In [32]:
import os
import ast
from collections import deque

In [131]:
game_folders = ['MO/', 'PLAY/', 'LIU/']
folder_name = 'output2017 2/'
all_game_files = []

for game_folder in game_folders:
    for file_name in os.listdir(folder_name + game_folder):
        all_game_files.append(folder_name + game_folder + file_name)

In [132]:
import numpy as np
from collections import Counter

w_dict = {'W' + str(i+1): i for i in range(9)} #万
b_dict = {'B' + str(i+1): i+9 for i in range(9)} #饼
t_dict = {'T' + str(i+1): i+18 for i in range(9)} #条
f_dict = {'F' + str(i+1): i+27 for i in range(4)} #风 东南西北
j_dict = {'J' + str(i+1): i+31 for i in range(3)} #（剑牌）中发白
h_dict = {'H' + str(i+1): i+34 for i in range(8)} #梅兰竹菊
total_dict = {**w_dict, **b_dict,**t_dict,**f_dict,**j_dict,**h_dict}

def serialize(array):
    res=np.zeros((4,42),int)
    count = Counter(array)
    for i in count:
        index = total_dict[i]
        nums = count[i]
        for j in range(nums):
            res[j][index] = 1
    return np.expand_dims(np.array(res), axis = 0)


def serialize_y(tile):
    res=np.zeros((42),int)
    index = total_dict[tile]
    res[index] = 1
    return res

In [170]:
def generate_training_set(file_name, player_num, history_num = 4):
    other_player = [i for i in range(4)]
    other_player.pop(player_num)
    
    master_X = np.empty((0,30, 4, 42), int)
    master_Y = np.empty((0,42), int)
    
    f = open(file_name, "r")
    title = f.readline()
    game_info = f.readline()
    wind = game_info.split('\t')[0]
    score = game_info.split('\t')[1]
    discard_card = [[],[],[],[]]
    past_game_history = deque([np.zeros((6, 4, 42),int) for x in range(history_num)])

    # Get player starting tile infromation
    for i in range(4):
        player_hand = f.readline()
        start = player_hand.find('[')
        end = player_hand.find(']') + 1
        starting_tile = ast.literal_eval(player_hand[start:end])
        if i == player_num:
            player_tile = starting_tile


    round_info = f.readline()
    
    while round_info != None and len(round_info) > 0:
        try:
            # Extracting information of that round
            round_info = round_info.split('\t')
            round_player_num = int(round_info[0])
            action = round_info[1]
            round_tile = ast.literal_eval(round_info[2])[0]
            eat_tile = round_info[3]

            # Not yet implemneted for target
            # Everyone discard tile can be seen for everyone
            # Now we have feature (X)
            if action == '打牌':
                if player_num == round_player_num:
                    player_tile.remove(round_tile)
                    Y = serialize_y(round_tile)
                discard_card[round_player_num].append(round_tile)

            elif action == '摸牌' or action == '补花后摸牌':
                if player_num == round_player_num:
                    player_tile.append(round_tile)
                else:
                    pass

            elif action == '补花':
                if player_num == round_player_num:
                  # Add in the flower list
                    pass
                else:
                    pass

            elif action == '吃':
                if player_num == round_player_num:
                    player_tile.append(eat_tile)
                else:
                    pass

            round_info = f.readline()

            # If round player number = your player number
            if round_player_num == player_num:
                # Feature encoding
                # Encoding own hand feature
                X = serialize(player_tile) 

                for discard_list in discard_card:
                    # Encoding all the discard card
                    X = np.concatenate((X, serialize(discard_list)), axis = 0) 

                # Encoding the current card
                X = np.concatenate((X, serialize([round_tile])), axis = 0) 

                X_with_history = np.copy(X)

                for past_game_list in past_game_history:
                    X_with_history = np.concatenate((X_with_history, past_game_list), axis = 0) # Encode the past history situation into feature

                past_game_history.append(X) # Append the current situation into the history queue
                past_game_history.pop() # Remove the last information history

#                 print(master_X.shape)
#                 print(X_with_history.shape)
                if action == '打牌':
                    master_X = np.append(master_X, np.expand_dims(X_with_history, axis = 0), axis = 0)
                    master_Y = np.append(master_Y, np.expand_dims(Y, axis = 0), axis = 0)

        except:
            break
            
    f.close()
    return master_X, master_Y

In [171]:
master_X = np.empty((0, 30, 4, 42), int)
master_Y = np.empty((0, 42), int)
file_count = 0


for file_name in all_game_files:
    #for player_num in range(0,4):
    X, Y = generate_training_set(file_name, player_num=0, history_num = 4)
    master_X = np.append(master_X, X, axis = 0)
    master_Y = np.append(master_Y, Y, axis = 0)
    if master_X.shape[0] > 10000:
        break
    print('Shape', master_X.shape[0])

Shape 14
Shape 24
Shape 34
Shape 47
Shape 53
Shape 64
Shape 78
Shape 96
Shape 117
Shape 132
Shape 150
Shape 165
Shape 179
Shape 190
Shape 209
Shape 215
Shape 234
Shape 243
Shape 259
Shape 269
Shape 282
Shape 287
Shape 292
Shape 308
Shape 328
Shape 344
Shape 357
Shape 364
Shape 376
Shape 386
Shape 400
Shape 417
Shape 426
Shape 435
Shape 446
Shape 458
Shape 470
Shape 486
Shape 494
Shape 505
Shape 516
Shape 531
Shape 542
Shape 564
Shape 577
Shape 585
Shape 593
Shape 607
Shape 620
Shape 627
Shape 636
Shape 648
Shape 659
Shape 665
Shape 684
Shape 705
Shape 722
Shape 733
Shape 746
Shape 755
Shape 766
Shape 777
Shape 786
Shape 797
Shape 816
Shape 833
Shape 840
Shape 855
Shape 872
Shape 881
Shape 891
Shape 909
Shape 921
Shape 930
Shape 939
Shape 951
Shape 963
Shape 979
Shape 988
Shape 1000
Shape 1013
Shape 1024
Shape 1040
Shape 1057
Shape 1066
Shape 1069
Shape 1079
Shape 1091
Shape 1102
Shape 1114
Shape 1125
Shape 1138
Shape 1151
Shape 1170
Shape 1180
Shape 1193
Shape 1197
Shape 1215
Shape 122

Shape 9129
Shape 9150
Shape 9162
Shape 9179
Shape 9192
Shape 9208
Shape 9220
Shape 9239
Shape 9254
Shape 9268
Shape 9278
Shape 9298
Shape 9310
Shape 9316
Shape 9332
Shape 9348
Shape 9355
Shape 9365
Shape 9378
Shape 9387
Shape 9396
Shape 9407
Shape 9426
Shape 9439
Shape 9453
Shape 9465
Shape 9475
Shape 9490
Shape 9502
Shape 9516
Shape 9526
Shape 9538
Shape 9546
Shape 9555
Shape 9565
Shape 9580
Shape 9597
Shape 9605
Shape 9623
Shape 9639
Shape 9655
Shape 9671
Shape 9686
Shape 9701
Shape 9719
Shape 9729
Shape 9740
Shape 9749
Shape 9760
Shape 9774
Shape 9783
Shape 9795
Shape 9807
Shape 9815
Shape 9817
Shape 9828
Shape 9840
Shape 9851
Shape 9870
Shape 9887
Shape 9900
Shape 9913
Shape 9930
Shape 9947
Shape 9968
Shape 9977
Shape 9987
Shape 9998


In [176]:
np.save('./input_X.npx', master_X)
np.save('./input_Y.npx', master_Y)